In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

In [4]:
tf.__version__

'2.16.2'

In [6]:

train_datagen = ImageDataGenerator(
    rescale = 1./255, # 신경망이 이미지 데이터 픽셀값 0부터 255 값을 잘 처리 못할 수도 있어서 0부터 1 사이의 값으로 만들기 위해서 255로 나눠준 것이다.
    shear_range = 0.2, # shear (전단) 변환 : 이미지를 일정 각도로 밀어 왜곡하는 변환이다. x축에서 밀수도 있고 y축에서 밀 수도 있다.
    zoom_range = 0.2,
    horizontal_flip = True
)


In [8]:
train_set = train_datagen.flow_from_directory ( # flow_from_directory 함수는 디렉토리 구조에서 이미지 로드한다.
    'dataset/training_set',
    target_size=(64,64),
    batch_size = 32,
    class_mode = 'categorical' # 다중 분류를 위한 categorical 모드
)

Found 854 images belonging to 9 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale= 1./255)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'categorical'
)


Found 369 images belonging to 9 classes.


In [12]:

cnn = tf.keras.models.Sequential()
# Sequential 모델은 층을 순차적으로 쌓아 올리는 단순한 모델 구조를 정의할 때 사용된다.
# 순차적으로 층을 추가하고, 데이터가 각 층을 통과하며 변환되도록 한다.

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size =3, activation = 'relu', input_shape=(64,64,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=9, activation='softmax')) # 클래스 수에 맞게 유닛 수를 9로 설정하고 softmax activate function 활성화 함수 사용



C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Adam 옵티마이저에 학습률 지정

learning_rate = 0.0008
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

cnn.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit( x = train_set, validation_data = test_set, epochs=60)

Epoch 1/60


C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2313 - loss: 2.0734 - val_accuracy: 0.2249 - val_loss: 2.0067
Epoch 2/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 33s 986ms/step - accuracy: 0.3334 - loss: 1.9131 - val_accuracy: 0.2900 - val_loss: 1.9901
Epoch 3/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 32s 972ms/step - accuracy: 0.4059 - loss: 1.7006 - val_accuracy: 0.2791 - val_loss: 2.2298
Epoch 4/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 32s 965ms/step - accuracy: 0.5064 - loss: 1.5165 - val_accuracy: 0.3008 - val_loss: 2.2688
Epoch 5/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 31s 959ms/step - accuracy: 0.5548 - loss: 1.3422 - val_accuracy: 0.3523 - val_loss: 2.1249
Epoch 6/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 31s 955ms/step - accuracy: 0.5611 - loss: 1.2826 - val_accuracy: 0.3306 - val_loss: 2.5851
Epoch 7/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 31s 952ms/step - accuracy: 0.6425 - loss: 1.0610 - val_accuracy: 0.3415 - val_loss: 2.6462
Epoch 8/60
27/27 ━━━━━━━━━━━━━━━━━━━━ 32s 958ms/step - accuracy: 0.6451 - loss: 0.9902 - val_accuracy: 0.2602 -

In [27]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/PE드럼_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)



In [29]:

# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
PE드럼 파손


In [31]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/PE방호벽_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
PE방호벽 파손


In [33]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/PE안내봉_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
PE안내봉 파손


In [35]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/도로_갈라짐.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
도로 갈라짐


In [20]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/라바콘_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
라바콘 파손


In [21]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/시선유도봉_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
시선유도봉 파손


In [22]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/제설함_파손.jpg', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
제설함 파손


In [23]:

# 테스트 이미지 예측 (class 9개 모두 다 해봐야 함)
test_image1 = image.load_img ('dataset/single_prediction/PE입간판_파손.png', target_size=(64,64))


# PIL (Python Imaging Libary) 이미지 인스턴스 = 이미지 데이터를 다루기 위한 python 객체
# PIL을 넘파이 배열로 변환하기 왜냐? 신경망 모델 자체가 넘파이 배열을 입력으로 받기 때문이다~ PIL 이미지를 넘파이 배열로 변환해야 한다.


test_image1 = image.img_to_array(test_image1)


# 배열에 새로운 차원을 추가한다.
# 딥러닝 프레임워크에서 모델이 입력 데이터를 처리하는 방식이 그렇다. keras, tensorflow, pytorch 등등 모두 데이터를 배치 단위로 처리한다.
# 4차원 입력 배열의 필요성
# 딥러닝 모델에 입력되는 데이터의 형식 : (배치크기, 높이, 너비, 채널수)
# 배치 처리를 위해서 배열을 4차원으로 만들기 위해 사용된다. (배치 크기, 높이, 너비, 채널 수)

test_image1 = np.expand_dims(test_image1, axis=0)

result1 = cnn.predict(test_image1/255.0) # test image도 픽셀 값이 0부터 255 사이의 값이기 때문에, 이걸 정규화 해줘야 한다. 0부터 1 사이의 값으로 정규화 실시한다.
# result = (1,9) 1개의 이미지와 9개의 클래스

# 클래스 인덱스 확인하기

print(train_set.class_indices)  # class_indices 속성: 클래스 레이블과 디렉토리 이름 사이의 매핑을 반환한다.

# 예측 결과 출력하기

predicted_class = np.argmax(result1, axis=1) # 하나의 요소를 가진 배열이 된다.
class_labels = list(train_set.class_indices.keys()) # 딕셔너리의 key (pe드럼 파손, pe휀스 파손 등등 ... ) 만 추출해서 list 로 변환한다.
prediction = class_labels[predicted_class[0]] # predicted_class의 첫번째이자 유일한 요소를 가져온다.

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
{'PE드럼 파손': 0, 'PE방호벽 파손': 1, 'PE안내봉 파손': 2, 'PE입간판 파손': 3, 'PE휀스 파손': 4, '도로 갈라짐': 5, '라바콘 파손': 6, '시선유도봉 파손': 7, '제설함 파손': 8}
PE입간판 파손


In [59]:


!pip install --upgrade tensorflow keras


In [61]:
import tensorflow as tf


# keras 모델을 tflite (모바일에 적합한 형태) 로 변환하기 

# 1 . 모델 저장
cnn.save('cnn_krs_2.keras')

# 2. 모델 로드
cnn_keras_model = tf.keras.models.load_model('cnn_krs_2.keras')

# 3. 모델의 호출 함수를 tf.function으로 래핑
@tf.function
def wrapped_model(inputs):
    return cnn_keras_model(inputs)

# 4. TFLiteConverter를 사용하여 모델을 TFLite 형식으로 변환
converter = tf.lite.TFLiteConverter.from_concrete_functions([wrapped_model.get_concrete_function(tf.TensorSpec([None, 64, 64, 3], tf.float32))])
tf_cnnMdl = converter.convert()

# 5. TFLite 모델을 파일로 저장하기
with open('cnn_keras_mdl_2.tflite', 'wb') as f:
    f.write(tf_cnnMdl)
